# Spatial processing
This notebook will cover image processing in the spatial domain for the image processing part of TDT4195.

In [2]:
#Import statements
from matplotlib import pyplot as plt
import numpy as np
from ipywidgets import interact, interact_manual, FloatSlider
import ipywidgets as widgets

## Sampling and quantization

The world around us is analog or continous, while digital images are discrete. In order to produce a digital image from a continous signal we must do the two processes called sampling and quantization.

### Sampling
In image processing sampling is the process of reducing the continous spatial signal to a discrete set of pixels. Its effect can be illustrated by excessively undersampling an image and looking at the result. An undersampled image will appear pixelated. 

In [104]:
def sample(x):
    image = plt.imread("images/strawberry.jpg")[::2, ::2]
    x = 2**(8-x)
    plt.figure(figsize = (10,10))
    plt.imshow(image[::x, ::x])
    plt.show()
    
_ = interact(sample,x=widgets.IntSlider(min=1, max=8, step=1, value=0))

interactive(children=(IntSlider(value=1, description='x', max=8, min=1), Output()), _dom_classes=('widget-inte…

### Quantization
Whereas sampling is the process discretizising image in the spatial extent to a discrete set of pixels, quantization is the process of reducing the continous range of possible color intensity values to a discrete set of intensities. Often we use uint8 to represent color values. That gives us a range of 256 different color intensity levels. Other times we may require a higher integer precision for a more true representation, or we may reduce the number of possible intensity levels in order to compress the image.

In [109]:
def quantize(color_bits):
    image = plt.imread("images/strawberry.jpg")[::2, ::2]
    x = 2**(8-color_bits)
    plt.figure(figsize = (10,10))
    plt.imshow(image - image % x)
    plt.show()
    
_ = interact(quantize,color_bits=widgets.IntSlider(min=1, max=8, step=1, value=0))

interactive(children=(IntSlider(value=1, description='color_bits', max=8, min=1), Output()), _dom_classes=('wi…

## Intensity transformations
Some of the simplest image processing operations are intensity transformations or point processing operations. Intensity transformations are image operations that operate on each pixel individually without respect to the neighbouring pixels. This is in contrast to neighbourhood processing, which assesses the neighbourhood around each pixel in order to transform it.

In intensity transforms each pixel is assigned a new color according to a scalar function:
$$c' = f(c)$$

The function f can vary alot depending on what we want to do. The following code snippets display some common mathematical operations that we can apply to images.

### Image historgram
When we apply intensity transforms we are usually interested in how the transformation affectes the image histogram. An image histogram is simply a histogram that displays the image distribution over all intensity values. We can calculate and display image histograms with matplotlib.

We can get a lot of usefull information from assessing the image histogram. Images where most values are at the lower end of the distribution are dark, images with mostly high values are bright, images intensity values spread over a short span have low contrast and images with values spread over all possible intensities are high in contrast. Ideally we want a flat image-histogram.

In [100]:
def display_image_and_histogram(image_path):
    image = plt.imread("images/bw_" + image_path + ".jpg")[::2,::2,0]
    plt.figure(figsize = (12,4))
    plt.subplot(1,2,1)
    plt.imshow(image, cmap = "gray")
    plt.subplot(1,2,2)
    
    #Always remeber to flatten the image array before calculating the histogram
    _ = plt.hist(image.reshape(-1), bins=64)
    plt.title("Image histogram")
    plt.show()


_ = interact(display_image_and_histogram, image_path = ["lake", "mountain", "alpine", "sunset", "boat"] )

interactive(children=(Dropdown(description='image_path', options=('lake', 'mountain', 'alpine', 'sunset', 'boa…

### Intuition - Window and level transform
We can transform an image's brightness and contrast by transforming the image-historgram. A simple method is the window/level-transform.

In [99]:
def make_window_level_transform(window, level):
    half_window = window/2
    y = np.zeros(256)
    for x in range(256):
        if(x < level - half_window):
            y[x] = 0
        elif (x >= level + half_window):
            y[x] = 255
        else:
            y[x] = (x - level + half_window) * 255/window
    return y.astype(np.uint8)

def window_transform_image(image_path, window, level):
    transform = make_window_level_transform(window, level)
    image = plt.imread("images/bw_" + image_path + ".jpg")[::2,::2,0]
    image = transform[image[:,:]]
    plt.figure(figsize=(18,6))
    plt.subplot(1,2,1)
    image[0,0] = 254   #Necessary so that matplotlib doesn't normalize all white images to black
    plt.imshow(image, cmap = "gray")
    plt.subplot(1,2,2)
    plt.plot(np.arange(256), transform)
    plt.title("Transformation function")
    plt.show()
    
    _ = plt.hist(image.reshape(-1), bins = 64)
    plt.title("Image histogram")
    plt.show()
    
_ = interact_manual(window_transform_image, image_path = ["lake", "mountain", "alpine", "sunset", "boat"],\
                    window = widgets.IntSlider(min = 0, max = 256, step = 2, value=256),\
             level = widgets.IntSlider(min = 0, max = 256, step = 1, value=127))

interactive(children=(Dropdown(description='image_path', options=('lake', 'mountain', 'alpine', 'sunset', 'boa…

### Gamma transform
Interactive display of how the gamma-transform affects an image.

In [12]:
def gamma_transform(gamma):
    image = plt.imread("images/bw_lake.jpg")[::2,::2,0]  
    image = image/255.
    image = image**gamma
    plt.figure(figsize = (8,4))
    plt.imshow(image, cmap = "gray")
    plt.show()
    image = (image * 255).astype(np.uint8).reshape(-1)
    plt.figure(figsize = (12,4))
    plt.subplot(1,2,1)
    _ = plt.hist(image, bins=64)
    plt.title("Image histogram")
    plt.subplot(1,2,2)
    x = np.arange(256)
    y = ((x/255.)**gamma)*255
    plt.plot(x,y)
    plt.title("Transformation function")
    plt.show()

_ = interact_manual(gamma_transform,  gamma=FloatSlider(min=0.01, max=5.0, step=1e-2, value=1.0))

interactive(children=(FloatSlider(value=1.0, description='gamma', max=5.0, min=0.01, step=0.01), Button(descri…

### Log and inverse log transform
Interactive display of the log and inverse log transform

In [14]:
def log_transform(transform):
    image = plt.imread("images/bw_lake.jpg")[:,:,0]  
    image = image/255.
    if transform == "log":
        c = 1/np.log(2)
        image = c*np.log(image + 1)
    elif transform == "inverse log":
        c = 1/np.log(2)
        image = np.exp(image/c) - 1
    else:
        pass
    plt.figure(figsize=(12,6))
    plt.imshow(image, cmap = "gray")
    plt.show()
    image = (image * 255).astype(np.uint8).reshape(-1)
    plt.figure(figsize=(10,6))
    _ = plt.hist(image, bins=64)
    plt.title("Image histogram")
    plt.show()

_ = interact_manual(log_transform, transform = ["log", "identity", "inverse log"])

interactive(children=(Dropdown(description='transform', options=('log', 'identity', 'inverse log'), value='log…

### Histogram equalization
Histogram equalization is a method used in image processing to improve contrast in images. It stretches the dynamic range for the most frequent intensity values, effectively spreading them out over the whole range of possible values. Normally this will imporve the contrast of images, brighten dark images and darken bright images.

In [92]:
def histogram_eq(image):
    #First we calculate the histogram.
    hist, bins = np.histogram(image[:,:], 256)
    
    #Then we calculate the cumulative histogram
    cumulative_hist = np.zeros((256), dtype = np.int)
    cumulative_hist[0] = hist[0]
    for i in range(1, 256):
        cumulative_hist[i] = cumulative_hist[i-1] + hist[i]
    
    #Then we normalize the histogram to [0., 1.]
    normalized_cumulative_hist = cumulative_hist.astype(float)
    normalized_cumulative_hist /= normalized_cumulative_hist[-1]
    
    #Then we construct an intensity mapping table by multiplying with the maximum intensity value
    table = (normalized_cumulative_hist * 255).astype(np.uint8)
    
    #Lastly we create the new image by mapping each of the old intensities to the new corresponding intensity
    new_im = table[image[:,:]]
    return new_im, table

def histogram_transform_image(image):
    new_image, transform = histogram_eq(image)
    plt.figure(figsize=(16,6))
    plt.subplot(1,2,1)
    plt.imshow(image, cmap = "gray")
    plt.title("Original image")
    plt.subplot(1,2,2)
    plt.imshow(new_image, cmap = "gray")
    plt.title("Transformed image")
    plt.show()
    plt.figure(figsize=(16,3))
    plt.subplot(1,3,1)
    plt.plot(np.arange(256), transform)
    plt.title("Transformation function")
    
    plt.subplot(1,3,2)
    _ = plt.hist(image.reshape(-1), bins = 64)
    plt.title("Original image histogram")
    
    plt.subplot(1,3,3)
    _ = plt.hist(new_image.reshape(-1), bins = 64)
    plt.title("Transformed image histogram")
    plt.show()

    
def window_level_then_histogram(image_name, window, level):
    window_level_transform = make_window_level_transform(window, level)
    image = plt.imread("images/bw_" + image_name + ".jpg")[:,:,0]
    image = window_level_transform[image]
    histogram_transform_image(image)
    
    
def gamma_then_histogram(image_name, gamma):
    image = plt.imread("images/bw_" + image_name + ".jpg")[:,:,0]
    image = (image/255)**gamma
    image = (image*255).astype(np.uint8)
    histogram_transform_image(image)
#_ = interact_manual(window_level_then_histogram, image_name = ["lake", "mountain", "alpine", "sunset", "boat"],\
#                   window = widgets.IntSlider(min = 0, max = 256, step = 2, value=256),\
#                   level = widgets.IntSlider(min = 0, max = 256, step = 1, value=127))


_ = interact_manual(gamma_then_histogram, image_name = ["alpine", "sunset", "boat", "lake", "mountain", ],\
                    gamma=FloatSlider(min=0.01, max=5.0, step=1e-2, value=1.0))

interactive(children=(Dropdown(description='image_name', options=('alpine', 'sunset', 'boat', 'lake', 'mountai…

## Neighborhood filtering
Neighborhood filtering covers image processing techniques that transform each pixel with respect to its neighbors. The common operation is image convolution. By varying the convolution kernels we can achieve many different effects. 

This tool is supposed to be an interactive display of different convolution kernels. Most of the kernels can be found [here](https://en.wikipedia.org/wiki/Kernel_(image_processing)). Common operations are image smoothening, image sharpening and edge detection. Normally, we apply image smoothening before edge detection. This is because edge detection is basically the same as derivating the image, and taking the derivative of a noisy signal will yield even more noise. If we first smoothen the image, we will eliminate some of the noise before applying the edge detection. Smoothening with a small kernel will allow you to detect fine edges, while applying a large smoothening kernel will allow you to detect only the cleares edges.

Try different combinations of covolution kernels and assess the results. Convolving the **boat** image with **box_9** and **sobel_y** is a good place to start. Have fun!

In [97]:
def convolve_image(image, kernel):
    h, w = image.shape[:2]
    kernel_size= kernel.shape[0]
    
    #Rotate 180 degrees in order to perform convolution and not correlation
    kernel = np.rot90(np.rot90(kernel))
    pad = (kernel_size - 1) // 2
    
    image = np.pad(image, pad_width = ((pad,pad), (pad,pad)), mode = "constant")
    
    output = np.zeros_like(image)
    for y in range(h):
        for x in range(w):
            im_slice = image[y: y+kernel_size, x: x + kernel_size]
            
            value = np.sum(im_slice*kernel, axis = (0,1))
            output[y + pad, x + pad] = value

    return output

kernel_map = {
    "identity"   : np.array([[0,0,0], [0,1,0], [0,0,0]]),
    "gaussian_3" : np.array([[1,2,1], [2,4,2], [1,2,1]])/16,
    "gaussian_5" : np.array([[1,4,6,4,1], [4,16,24,16,4],\
                             [6,24,36,24,6], [4,16,24,16,4], [1,4,6,4,1]]) /256,
    "gaussian_7" : np.array([[0,0,1,2,1,0,0], [0,3,13,22,13,3,0], [1,13,59,97,69,13,1],\
                            [2,22,97,159,97,22,2], [1,13,59,97,59,13,1], [0,3,13,22,13,3,0],\
                            [0,0,1,2,1,0,0]])/1003,
    "box_3"      : np.ones((3,3))/9,
    "box_5"      : np.ones((5,5))/25,
    "box_7"      : np.ones((7,7))/49,
    "box_9"      : np.ones((9,9))/81,
    "edge_1"     : np.array([[1,0,-1], [0,0,0],[-1,0,1]]),
    "edge_2"     : np.array([[0,-1,0], [-1,4,-1], [0,-1,0]]),
    "edge_3"     : np.array([[-1,-1,-1], [-1,8,-1], [-1,-1,-1]]),
    "sharpen"    : np.array([[0,-1,0], [-1,5,-1], [0,-1,0]]),
    "sobel_x"    : np.array([[1,0,-1], [2,0,-2], [1,0,-1]]),
    "sobel_y"  : np.array([[1,2,1], [0,0,0], [-1,-2,-1]])
    }

def display_convolved_image(image_name, kernel_1, kernel_2):
    image = plt.imread("images/bw_" + image_name + ".jpg")[:,:,0]
    if image_name in ["alpine", "sunset", "boat", "lake", "mountain"]:
        image = image[::2,::2] 


    output = convolve_image(image, kernel_map[kernel_1])
    output = convolve_image(output, kernel_map[kernel_2])
    plt.figure(figsize = (10,10))
    plt.imshow(output, cmap = "gray")            
    plt.show()     

_ = interact_manual(display_convolved_image, image_name = ["butterfly","alpine", "boat", "lake", "mountain"],\
                    kernel_1 = kernel_map.keys(), kernel_2 = kernel_map.keys())
    

interactive(children=(Dropdown(description='image_name', options=('butterfly', 'alpine', 'boat', 'lake', 'moun…